In [ ]:

import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl import workbook
import xlrd
import os
import numpy as np
import datetime as dt

# file pathes to b2w report "Cost Detail With Job Cost ID" and to where it should get saved once modified 
new_file_Path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID1.xlsx"
og_path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID.xlsx"
# list of lists containg reference file paths, the column to search, and the category that should be assigned 
estimate_reference_paths = [[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_pay_item_db.xlsx"), "name", "Pay Item"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_task_db.xlsx"), "Task Name", "Task"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_crew_db.xlsx"), "Crew Name", "Crew"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_materials_db.xlsx"),"Material Type", "Material"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_misc_db.xlsx"),"Name", "Misc."],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_labor_db.xlsx"), "Labor Type", "Labor"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_equipment_db.xlsx"),"Equipment Name", "Equpment"]]

# Read excel workbook of Cost Detail With Job Cost ID
df1 = pd.read_excel(og_path, skiprows=12)
# Remove blank columns/rename columns/dropna
df2 = df1.drop(df1.columns[[2,6,7,8,9,10,11,13,14,16]], axis=1)
df2.rename(columns={"Unnamed: 1":"Pay Item", "Unnamed: 3":"Task/Componenet", "Unnamed: 4":"Crew/Componenet", "Unnamed: 5":"Labor/Equipment",
                          "Direct Cost":"Unit Direct Cost","Direct Cost.1":"Total Direct Cost"},inplace=True)
df2.dropna(subset="Unit Direct Cost", inplace=True)
df3 = df2[df2["Unit Direct Cost"].str.contains("$")]
# fill blank pay items with item above for hiararchy
df3['Pay Item'] = df3['Pay Item'].ffill()
df3[["Category","Hierarchy #", "Hierarchy Structure"]] = "Custom","",""
# establish categories and hierarchy 
for index, row1 in df3.iterrows():
    hierarchy = 0 
    for col1 in range(1,5):
        hierarchy += 1
        if pd.notna(row1.iloc[col1]):
            no_bracket_value = str(row1.iloc[col1]).split(" [")[0]
            # print(no_bracket_value)
            for cat in estimate_reference_paths:

# need a method to identify crews as crews 

                # print(row1.iloc[col1])
                # if cat[2] == "Crew":
                #     no_bracket_value = no_bracket_value.split("(")[0]
                if no_bracket_value in list(cat[0][cat[1]]):
                    df3.at[index,"Category"] = cat[2]
                    df3.at[index,"Hierarchy #"] = hierarchy



In [26]:
# df3[df3['Hierarchy #'] == 1]
df3.head(25)

,Description,Pay Item,Task/Componenet,Crew/Componenet,Labor/Equipment,Job Cost ID,Quantity,UM,Unit Direct Cost,Total Direct Cost,Category,Hierarchy #,Hierarchy Structure
0,NaN,AD - Area Drain / Inlet,NaN,NaN,NaN,NaN,0.00,EACH,$0.00,$0.00,Pay Item,1,
1,NaN,AD - Area Drain / Inlet,080.370. Area Inlet,NaN,NaN,080.370.,0.00,EACH,$0.00,$0.00,Task,2,
2,NaN,AD - Area Drain / Inlet,NaN,"Storm Sewer Manhole / Inlets (2.00 PH/EACH, 0.00",NaN,NaN,0.00,EACH,$0.00,$0.00,Pay Item,1,
4,NaN,AD - Area Drain / Inlet,NaN,NaN,Excavator Cat 349/Kom 400-490,NaN,0.00,HR,$98.74,$0.00,Equpment,4,
5,NaN,AD - Area Drain / Inlet,NaN,NaN,Loader Cat 938 - 950,NaN,0.00,HR,$50.67,$0.00,Equpment,4,
6,NaN,AD - Area Drain / Inlet,NaN,NaN,"Watertruck 4,000 Gal Water Truck",NaN,0.00,HR,$35.22,$0.00,Equpment,4,
7,NaN,AD - Area Drain / Inlet,NaN,NaN,Pickup 3/4 Ton Pickup,010.230.,0.00,HR,$18.74,$0.00,Equpment,4,
8,NaN,AD - Area Drain / Inlet,NaN,NaN,Small Tools,NaN,0.00,HR,$5.00,$0.00,Equpment,4,
9,NaN,AD - Area Drain / Inlet,NaN,NaN,Operator Utilities [2],NaN,0.00,HR,$0.00,$0.00,Labor,4,
10,NaN,AD - Area Drain / Inlet,NaN,NaN,Labor Utility [3],NaN,0.00,HR,$0.00,$0.00,Labor,4,


In [ ]:
# flag items with same cost ID but different Ums that are not materials or misc
# flag potentially wrong Cost ID when compared to Cost Id reference sheet from Jill 
# Flag materials inside tasks
# flag materials that do not have Cost IDs
# flag items that have UM of Day/Month/LS that a foreman can charge to 
# flag potentially custom Items, they may need complete restructure
# flag pay items that have taskes with different work types
